<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/multifit_es_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 18.4MB/s 
     |████████████████████████████████| 3.0MB 20.5MB/s 
     |████████████████████████████████| 890kB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=01b7083f265d977a13c4c3aaa58c9c78615bb7b8af0aa720fb949b128022d89c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
pip install git+https://github.com/n-waves/multifit

  Cloning https://github.com/n-waves/multifit to /tmp/pip-req-build-l_rgniix
  Running command git clone -q https://github.com/n-waves/multifit /tmp/pip-req-build-l_rgniix
  Created wheel for multifit: filename=multifit-1.0-cp36-none-any.whl size=24506 sha256=bdca1105c6fe0b83673b02cd900d8d130430a1794d77aa4979ba3e7526c534c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-sx8ity3l/wheels/b7/92/1e/246f31a4e84fd665b5907cb96765f696be45e814ff68a5fd4a
Successfully built multifit


In [4]:
pip install sacremoses

In [5]:
pip install sentencepiece

In [6]:
from zipfile import ZipFile 
path = F"/content/gdrive/My Drive/multilingual toxic/" 
with ZipFile(path+'toxic.zip', 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')
import pandas as pd
def load_data():
    trn=pd.read_csv('jigsaw-toxic-comment-train.csv',usecols=['toxic','comment_text'])
    val=pd.read_csv('validation.csv',usecols=['toxic','comment_text','lang'])
    tst=pd.read_csv('test.csv',usecols=['lang','content'])  
    sub=pd.read_csv('submission (1).csv')  
    return trn,tst,val,sub
train,test,valid,sub=load_data()
test['toxic']=sub['toxic'].round()

File Name                                             Modified             Size
jigsaw-toxic-comment-train.csv                 2020-06-08 23:38:40     95538001
test.csv                                       2020-06-08 23:39:02     28783206
validation.csv                                 2020-06-08 23:38:52      3178555
submission (1).csv                             2020-06-08 23:50:22      1161879
Extracting all the files now...
Done!


In [7]:
print(train.shape,test.shape,valid.shape,sub.shape)

(223549, 2) (63812, 3) (8000, 3) (63812, 2)


In [8]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import *
from fastai.text import *
import multifit
exp = multifit.from_pretrained("es_multifit_paper_version")
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
fa_config =  exp.pretrain_lm.tokenizer.get_fastai_config(add_open_file_processor=True)

In [9]:
es=test.loc[test['lang']=='es']
es['id']=es.index
es=es.reset_index(drop=True)
es.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,content,lang,toxic,id
0,el skate es unos de los deportes favoritos de ...,es,0.0,7
1,Me doy la bienvenida. A este usuari le gusta c...,es,0.0,8
2,"ES NOTABLEMENTE TENDENCIOSO, NO SE HABLA DE CU...",es,0.0,9
3,"El Jardín de infantes Nº938, fundado en 1989, ...",es,0.0,22
4,Daré explicaciones y/o aclaraciones a cualquie...,es,0.0,49


In [10]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.127.217.250:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.127.217.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.127.217.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [11]:
es=es.loc[:100]
es['toxic']=es['toxic'].astype(np.long)
ln=es.shape[0]//5
trn=es.loc[ln:]
val=es.loc[:ln]
print(trn.shape,val.shape)

(81, 4) (21, 4)


In [12]:
with strategy.scope():
    exp = multifit.from_pretrained("es_multifit_paper_version")
    fa_config =  exp.pretrain_lm.tokenizer.get_fastai_config(add_open_file_processor=True)
    data_lm = (TextList.from_df(es, **fa_config)
            .split_by_rand_pct(0.1)
            .label_for_lm()           
            .databunch(bs=32))
    learn = exp.finetune_lm.get_learner(data_lm)  
    # learn is a preconfigured fastai learner with a pretrained model loaded
data_lm.show_batch()

Training args:  {'drop_mult': 0.3, 'true_wd': False, 'wd': 1e-07, 'pretrained': False, 'clip': 0.12} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15, 'tie_weights': True, 'out_bias': True}


/usr/local/lib/python3.6/dist-packages/setuptools/distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Setting LM training seed seed to 0
Loading pretrained weights:  [PosixPath('/root/.fastai/models/es_multifit_paper_version/lm_best'), PosixPath('/root/.fastai/models/es_multifit_paper_version/itos')]


/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,"us ua ri ▁le ▁gu sta ▁confronta rse ▁con ▁los ▁ignora ntes ▁. ▁ ¿ ▁ xxmaj ▁va mos ▁a ▁estar ▁ tú ▁y ▁y o ▁de ▁acuerdo ▁o ▁no ▁ ? ▁( ▁discusión ▁) ▁18 ▁: ▁28 ▁27 ▁ dic ▁2014 ▁( ▁ xxup ▁ ut c ▁) ▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁es ▁ xxup ▁notable mente ▁ xxup ▁ ten den ci oso ▁, ▁"
1,"▁a s . lu is ▁ xxmaj ▁ bla u gen - ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁ da ré ▁explica ciones ▁y ▁/ ▁o ▁aclara ciones ▁a ▁cualquier ▁otro ▁usuario ▁, ▁excepto ▁a ▁un ▁usuario ▁ arro ga nte ▁y ▁con ven en cier o ▁como ▁tu ▁ xxmaj ▁ni hil o ▁. ▁ xxmaj ▁salud os ▁. ▁ ★ ▁ha ck - master ▁ ★ ▁("
2,xxbos ▁ xxfld ▁1 ▁una ▁persona ▁gor da ▁es ▁a k ella ▁ k ▁le ▁so bra ▁mucho ▁peso ▁y ▁ k ▁su ▁piel ▁es ▁tan ▁fo fa ▁ k ▁se ▁le ▁de s mont a ▁por ▁los ▁cartucho s ▁fibra op ticos ▁ya ▁ k ▁tu ▁gran ▁gran o ▁del ▁ culo ▁ te ▁ pesa ▁mas ▁ k ▁los ▁test i ku los ▁ k ▁ te ▁ku
3,"▁o ▁no ▁le ▁ da ▁la ▁gana ▁entender ▁ ? ▁, ▁es ▁demasiado ▁el ▁tener ▁que ▁li di ar ▁con ▁una ▁persona ▁in ma dura ▁que ▁ pone ▁es tu pi de ces ▁ cre y endo ▁que ▁es ▁un ▁juego ▁. ▁y ▁lo ▁ v uel vo ▁a ▁re pe tir ▁la ▁próxima ▁vez ▁no ▁hab rá ▁próxima ▁. ▁ xxbos ▁ xxfld ▁1 ▁ xxup ▁no ▁ xxup"
4,"▁ edi tar ▁pi do ▁dis cul pas ▁por ▁las ▁mole sti as ▁que ▁e ▁causa do ▁y ▁por ▁favor ▁ arre gle n lo ▁ xxbos ▁ xxfld ▁1 ▁ xxmaj ▁ya ▁ ves ▁lo ▁que ▁consigue s ▁por ▁pasar me ▁por ▁donde ▁me ▁ pla z ca ▁y ▁no ▁por ▁el ▁for ro ▁el ▁o ja l ▁de ▁la ▁sola pa ▁que ▁va mos ▁, ▁pro ba do"


In [13]:
learn.fit_one_cycle(10,max_lr=1e-4)

epoch,train_loss,valid_loss,accuracy,time
0,6.989575,5.539819,0.168750,00:32
1,6.803756,5.460157,0.167411,00:31
2,6.891234,5.504026,0.166071,00:31
3,6.900005,5.516753,0.163393,00:31
4,6.931806,5.411703,0.167857,00:31


KeyboardInterrupt: ignored

In [14]:
self=exp.finetune_lm
CLS_BEST = 'cls_best'
LM_BEST = "lm_best"
ENC_BEST = "enc_best"
experiment_path=Path(path)
self.experiment_path=experiment_path
tokenizer = self.base.tokenizer
print("Experiment", experiment_path)

self.experiment_path = experiment_path
tokenizer.save(self.experiment_path, learn=learn)
learn.to_fp32()
learn.save_encoder('/content/'+ENC_BEST)
learn.save(LM_BEST, with_opt=False)
# learn.destroy()
self.save_paramters()
print("Language model saved to", self.experiment_path)

Experiment /content/gdrive/My Drive/multilingual toxic
Copy sp model from /root/.fastai/models/es_multifit_paper_version to /content/gdrive/My Drive/multilingual toxic
Saving dump to /content/gdrive/My Drive/multilingual toxic/finetuning.json
Language model saved to /content/gdrive/My Drive/multilingual toxic


In [15]:
self=exp.classifier
self.base.experiment_path=Path('')
from pathlib import Path
with strategy.scope():
    exp.classifier.experiment_path=Path('')
    data_clas = TextClasDataBunch.from_df("", trn.reset_index(drop=True),val,  text_cols = 'content', label_cols = 'toxic',bs=32,vocab=data_lm.vocab)
    data_clas.vocab.itos = data_lm.vocab.itos
    l1rn = exp.classifier.get_learner(data_clas)  
# learn is a preconfigured fastai learner with a pretrained model loaded


Using Label smoothing with eps =  0.05
Setting Classifier weights seed seed to 0
Training args:  {'drop_mult': 0.5, 'wd': 0.01, 'pretrained': False, 'bptt': 70, 'loss_func': FlattenedLoss of LabelSmoothingCrossEntropy(), 'clip': 0.12, 'silent': False} config:  {'emb_sz': 400, 'n_hid': 1550, 'n_layers': 4, 'pad_token': 1, 'qrnn': True, 'bidir': False, 'output_p': 0.25, 'hidden_p': 0.1, 'input_p': 0.2, 'embed_p': 0.02, 'weight_p': 0.15}
Loading pretrained model /content/enc_best
Setting Classifier training seed seed to 0


In [16]:
data_clas.show_batch()

text,target
"xxbos xxup xxunk xxup ya xxup xxunk , xxup me xxup xxunk xxup la xxup xxunk xxup de xxup mi xxup xxunk , xxup ya xxup que xxup ni xxup xxunk xxup xxunk xxup me xxup xxunk xxup xxunk xxup que xxup mi xxup xxunk xxup xxunk xxup xxunk xxup xxunk xxup xxunk xxup xxunk xxrep 4 o xxup por xxup xxunk xxup xxunk , xxup xxunk , xxup xxunk",0
"xxbos xxmaj en xxunk xxunk de xxmaj xxunk , xxunk de la xxunk xxup ser , se ha xxunk la xxunk xxunk y xxunk del xxunk xxunk xxunk xxunk de 80 xxunk y xxunk xxunk a xxunk xxunk por xxunk de xxunk xxunk ( se xxunk xxunk xxunk de xxunk . de xxunk ¿ xxup xxunk xxup pone xxup el xxup xxunk ? xxrep 6 . xxmaj la xxunk se",0
"xxbos xxunk es un xxunk , lo xxunk xxunk xxunk en xxunk e xxunk por xxunk , xxunk y xxunk xxmaj xxunk in xxmaj xxunk xxunk xxunk que xxunk xxunk > la xxunk xxunk xxunk que xxunk xxunk xxmaj de xxunk . xxmaj mal xxunk es xxunk , no xxunk ( al xxunk que se xxunk de las xxunk de xxunk ) xxunk el xxunk xxunk de xxunk y xxunk",0
"xxbos xxmaj el xxmaj xxunk xxmaj xxunk se xxunk de xxmaj xxunk en xxunk de xxunk , xxunk xxunk de xxunk tipo xxunk la xxunk xxunk xxunk que xxunk en la xxunk . xxmaj xxunk de xxunk xxunk xxunk xxunk xxunk xxunk en xxunk xxunk xxunk en los xxunk xxmaj xxunk . xxmaj esta xxunk xxunk xxunk xxunk de los xxunk xxunk por xxmaj xxunk xxunk xxunk una xxunk de",0
"xxbos y xxunk yo ya xxunk , que xxunk xxunk xxunk que xxunk que xxunk xxunk a xxunk xxunk , el xxunk xxunk xxunk , que xxunk xxunk el xxunk xxunk xxunk xxunk con el xxunk a la xxunk de xxunk y xxunk . xxunk y xxunk xxunk xxunk 4 xxunk xxunk de que xxunk xxunk xxunk , por lo que xxunk xxunk que xxunk xxunk xxunk que xxunk xxunk",0


In [17]:
l1rn.fit_one_cycle(10,max_lr=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.687279,0.656084,0.809524,00:12


In [18]:
pre=l1rn.get_preds()[0][:,1].reshape(-1,1)

In [21]:
val['toxic']=np.asarray(pre)
val.to_csv(path+'es_1.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
